# **1. Importing Necessary Files and Libraries**

In [1]:
# mounting drive to colab
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# importing python file containing CNN architecture
!cp "drive/My Drive/Facial Classification - Gender/CNN_Models/smallervggnet.py" .
!cp "drive/My Drive/Facial Classification - Gender/CNN_Models/cnn_first.py" .
from smallervggnet import SmallerVGGNet
from cnn_first import Simple_CNN

In [3]:
# importing the necessary packages
from sklearn.preprocessing import LabelBinarizer
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD
from keras.models import load_model
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import random
import pickle
import cv2
import os

# set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use('Agg')

# defining paths for saving results and image sizes
SAVE_PATH_VGG = 'drive/My Drive/Facial Classification - Gender/VGG_results'
SAVE_PATH_CNN = 'drive/My Drive/Facial Classification - Gender/SimpleCNN_results'
IMG_WIDTH = 64
IMG_HEIGHT = 64

# **2. Loading Images**

In [4]:
# initialize the data and labels
print('[INFORMATION] Loading images...')
data = []
labels = []

# grab the images and randomly shuffle them
DATASET_PATH = 'drive/My Drive/Facial Classification - Gender/dataset_genre'
imagePaths = sorted(list(paths.list_images(DATASET_PATH)))
random.seed(42)
random.shuffle(imagePaths)

# loop over the input images
for imagePath in imagePaths:
  # load the image, resize it to 64 x 64 pixels (the required input spatial dimensions of SmallerVGGNet),
  # and store it in the data list
  image = cv2.imread(imagePath)
  image = cv2.resize(image, (IMG_WIDTH, IMG_HEIGHT))
  data.append(image)

  # extract the class label from the image path and update the labels list
  label = imagePath.split(os.path.sep)[-2]
  labels.append(label)

print('[INFORMATION] Images Loaded and Labels Extracted sucessfully!')

[INFORMATION] Loading images...
[INFORMATION] Images Loaded and Labels Extracted sucessfully!


In [5]:
print('Dataset size: {} images of 2 labels'.format(len(data)))

Dataset size: 6770 images of 2 labels


# **3. Data Preprocessing and Data Augmentation**

In [6]:
# scale the raw pixel intensities to the range [0, 1]
data = np.array(data, dtype = 'float')/255.0
labels = np.array(labels)

# partition the data into training and testing splits using 75 % of the data for training and the remaining 25 % for testing
(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size = 0.25, random_state=42)

# convert the labels from strings to integers
lb = LabelBinarizer()
trainY = lb.fit_transform(trainY)
testY = lb.fit_transform(testY)

# now, convert the labels from integers to vectors
trainY = to_categorical(trainY, num_classes=2, dtype='float32')
testY = to_categorical(testY, num_classes=2, dtype='float32')

# construct image generator for data augmentation
aug = ImageDataGenerator(rotation_range=30, width_shift_range=0.1,
                         height_shift_range = 0.1, shear_range = 0.2,
                         zoom_range = 0.2, horizontal_flip = True,
                         fill_mode = 'nearest')

# **4. First Model Initializing - VGG Architecture**

In [7]:
# initialize our VGG-lie Convolutional Neural Network
model = SmallerVGGNet.build(width=IMG_WIDTH, height=IMG_HEIGHT, depth=3, classes=len(lb.classes_))

# initialize our initial learning rage, # of epochs to train for and batch size
INIT_LR = 0.01
EPOCHS = 75
BATCH_SIZE = 32

# initialize the model and optimizer
print('[INFORMATION] Loading Neural Network Model...')
opt = SGD(lr = INIT_LR, decay = INIT_LR/EPOCHS)
model.compile(loss='categorical_crossentropy', optimizer = opt, metrics=['accuracy'])
print('[INFORMATION] Neural Network Model successfully loaded!\n')
model.summary()

[INFORMATION] Loading Neural Network Model...
[INFORMATION] Neural Network Model successfully loaded!

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 64, 64, 32)        896       
_________________________________________________________________
activation (Activation)      (None, 64, 64, 32)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 64, 64, 32)        128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 21, 21, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 21, 21, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 21, 21, 64)        18496     
___________________

# **5. First Model Training**

In [8]:
# train the network
print('[INFORMATION] Starting First Model Training...')
H = model.fit_generator(aug.flow(trainX, trainY, batch_size=BATCH_SIZE),
                        validation_data = (testX, testY),
                        steps_per_epoch = len(trainX)//BATCH_SIZE,
                        epochs = EPOCHS)
print('[INFORMATION] First CNN Model sucessfully trained')

[INFORMATION] Starting First Model Training...
Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/75
158/158 [==============================] - 74s 469ms/step - loss: 0.9361 - accuracy: 0.6202 - val_loss: 0.9367 - val_accuracy: 0.4985
Epoch 2/75
158/158 [==============================] - 74s 467ms/step - loss: 0.7180 - accuracy: 0.6714 - val_loss: 0.8888 - val_accuracy: 0.5304
Epoch 3/75
158/158 [==============================] - 74s 467ms/step - loss: 0.6211 - accuracy: 0.7064 - val_loss: 0.6471 - val_accuracy: 0.6444
Epoch 4/75
158/158 [==============================] - 78s 496ms/step - loss: 0.5904 - accuracy: 0.7138 - val_loss: 0.6388 - val_accuracy: 0.6592
Epoch 5/75
158/158 [==============================] - 74s 467ms/step - loss: 0.5782 - accuracy: 0.7245 - val_loss: 0.5274 - val_accuracy: 0.7265
Epoch 6/75
158/158 [==============================] - 74s 467ms/step - loss: 0.5677 - accuracy: 0.7233 - val_loss: 0.5246 - val_accuracy: 0.7354
Epoch 7

# **6. First Model Evaluation**

In [10]:
# evaluate the network
print('[INFORMATION] Starting Neural Network Evaluation...')
predictions = model.predict(testX, batch_size = 32)
print(classification_report(testY.argmax(axis=1),
                            predictions.argmax(axis = 1),
                            target_names = lb.classes_))

# plot the training loss and accuracy curves
N = np.arange(0, EPOCHS)
plt.style.use('ggplot')
plt.figure(figsize=(20,20))
plt.plot(N, H.history['loss'], label = 'train_loss')
plt.plot(N, H.history['val_loss'], label = 'val_loss')
plt.plot(N, H.history['accuracy'], label = 'train_acc')
plt.plot(N, H.history['val_accuracy'], label = 'val_acc')
plt.title('Training Loss and Accuracy (SmallerVGGNet)')
plt.xlabel('Epoch #')
plt.ylabel('Loss / Accuracy')
plt.legend()
plt.show()
plt.savefig(SAVE_PATH_VGG + '/learning_curves.png')

# save the model and label binarizer to disk
print('[INFORMATION] Serializing Network and Label Binarizer...')
model.save(SAVE_PATH_VGG)

[INFORMATION] Starting Neural Network Evaluation...
              precision    recall  f1-score   support

         Man       0.86      0.94      0.90       849
       Woman       0.93      0.84      0.89       844

    accuracy                           0.89      1693
   macro avg       0.90      0.89      0.89      1693
weighted avg       0.90      0.89      0.89      1693

[INFORMATION] Serializing Network and Label Binarizer...
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: drive/My Drive/Facial Classification - Gender/VGG_results/assets


# **7. Second Model Initializing - Double Convolution Architecture**

In [11]:
# initialize our Simple Convolutional Neural Network
model = Simple_CNN.build(width=IMG_WIDTH, height=IMG_HEIGHT, depth=3, classes=len(lb.classes_))

# initialize our initial learning rage, # of epochs to train for and batch size
EPOCHS = 150
BATCH_SIZE = 32

# initialize the model and optimizer
print('[INFORMATION] Loading Neural Network Model...')
model.compile(loss='binary_crossentropy', optimizer = 'adam', metrics=['accuracy'])
print('[INFORMATION] Neural Network Model successfully loaded!\n')
model.summary()

[INFORMATION] Loading Neural Network Model...
[INFORMATION] Neural Network Model successfully loaded!

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 62, 62, 32)        896       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 31, 31, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 29, 29, 32)        9248      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 6272)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 1024)              6423552   
_________________

# **8. Second Model Training**

In [12]:
# train the network
print('[INFORMATION] Starting Second Model Training...')
H = model.fit_generator(aug.flow(trainX, trainY, batch_size=BATCH_SIZE),
                        validation_data = (testX, testY),
                        steps_per_epoch = len(trainX)//BATCH_SIZE,
                        epochs = EPOCHS)
print('[INFORMATION] Second CNN Model sucessfully trained')

[INFORMATION] Starting Second Model Training...
Epoch 1/150
158/158 [==============================] - 36s 227ms/step - loss: 0.6662 - accuracy: 0.6246 - val_loss: 0.5185 - val_accuracy: 0.7318
Epoch 2/150
158/158 [==============================] - 36s 226ms/step - loss: 0.5338 - accuracy: 0.7386 - val_loss: 0.4757 - val_accuracy: 0.7809
Epoch 3/150
158/158 [==============================] - 36s 225ms/step - loss: 0.5064 - accuracy: 0.7554 - val_loss: 0.4096 - val_accuracy: 0.8116
Epoch 4/150
158/158 [==============================] - 36s 226ms/step - loss: 0.4875 - accuracy: 0.7679 - val_loss: 0.4122 - val_accuracy: 0.8133
Epoch 5/150
158/158 [==============================] - 36s 225ms/step - loss: 0.4727 - accuracy: 0.7754 - val_loss: 0.3998 - val_accuracy: 0.8240
Epoch 6/150
158/158 [==============================] - 36s 225ms/step - loss: 0.4488 - accuracy: 0.7939 - val_loss: 0.3779 - val_accuracy: 0.8334
Epoch 7/150
158/158 [==============================] - 35s 224ms/step - loss

# **9. Second Model Evaluating**

In [13]:
# evaluate the network
print('[INFORMATION] Starting Neural Network Evaluation...')
predictions = model.predict(testX, batch_size = 32)
print(classification_report(testY.argmax(axis=1),
                            predictions.argmax(axis = 1),
                            target_names = lb.classes_))

# plot the training loss and accuracy curves
N = np.arange(0, EPOCHS)
plt.style.use('ggplot')
plt.figure(figsize=(10,10))
plt.plot(N, H.history['loss'], label = 'train_loss')
plt.plot(N, H.history['val_loss'], label = 'val_loss')
plt.plot(N, H.history['accuracy'], label = 'train_acc')
plt.plot(N, H.history['val_accuracy'], label = 'val_acc')
plt.title('Training Loss and Accuracy (Simple CNN)')
plt.xlabel('Epoch #')
plt.ylabel('Loss / Accuracy')
plt.legend()
plt.show()
plt.savefig(SAVE_PATH_CNN + '/learning_curve.png')

# save the model and label binarizer to disk
print('[INFORMATION] Serializing Network and Label Binarizer...')
LABEL_BIN = SAVE_PATH_CNN + '/binarizer.txt'
model.save(SAVE_PATH_CNN)

[INFORMATION] Starting Neural Network Evaluation...
              precision    recall  f1-score   support

         Man       0.93      0.93      0.93       849
       Woman       0.93      0.93      0.93       844

    accuracy                           0.93      1693
   macro avg       0.93      0.93      0.93      1693
weighted avg       0.93      0.93      0.93      1693

[INFORMATION] Serializing Network and Label Binarizer...
INFO:tensorflow:Assets written to: drive/My Drive/Facial Classification - Gender/SimpleCNN_results/assets


# **10. First Model Testing**

In [14]:
# load the desired model and label binarizer
print('[INFORMATION] Loading Neural Network Model and Label Binarizer')
model = load_model(SAVE_PATH_VGG)

# load images to predict
PREDICT_PATH = 'drive/My Drive/Facial Classification - Gender/dataset_test'
imagePaths = sorted(list(paths.list_images(PREDICT_PATH)))

plt.figure(figsize=(10,10))
plt.subplots_adjust(wspace = 0.2, hspace = 0.01)
k = 1
for imagePath in imagePaths:
  # load the image, resize it to 64 x 64 pixels, and store it in the data list
  image = cv2.imread(imagePath)
  output = image.copy()
  image = cv2.resize(image, (IMG_WIDTH, IMG_HEIGHT))

  # add the batch dimension
  image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))

  # make a prediction on the image
  preds = model.predict(image)

  # find the class label index with the largest corresponding probability
  i = preds.argmax(axis = 1)[0]
  if i == 0:
    label = 'Man'
  else:
    label = 'Woman'

  # draw the class label + probability on the output image
  text = '{}: {:.2f} %'.format(label, preds[0][i]*100)

  # show the output imges
  plt.subplot(2, 5, k)
  plt.imshow(output)
  plt.title(text, fontsize = 10)
  plt.grid(b=None)
  plt.axis('off')

  k += 1

plt.savefig(SAVE_PATH_VGG + '/predictions.png')

[INFORMATION] Loading Neural Network Model and Label Binarizer


# **11. Second Model Testing**

In [15]:
# load the desired model and label binarizer
print('[INFORMATION] Loading Neural Network Model and Label Binarizer')
model = load_model(SAVE_PATH_CNN)

# load images to predict
PREDICT_PATH = 'drive/My Drive/Facial Classification - Gender/dataset_test'
imagePaths = sorted(list(paths.list_images(PREDICT_PATH)))

plt.figure(figsize=(10,10))
plt.subplots_adjust(wspace = 0.2, hspace = 0.01)
k = 1
for imagePath in imagePaths:
  # load the image, resize it to 64 x 64 pixels, and store it in the data list
  image = cv2.imread(imagePath)
  output = image.copy()
  image = cv2.resize(image, (IMG_WIDTH, IMG_HEIGHT))

  # add the batch dimension
  image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))

  # make a prediction on the image
  preds = model.predict(image)

  # find the class label index with the largest corresponding probability
  i = preds.argmax(axis = 1)[0]
  if i == 0:
    label = 'Man'
  else:
    label = 'Woman'

  # draw the class label + probability on the output image
  text = '{}: {:.2f} %'.format(label, preds[0][i]*100)

  # show the output imges
  plt.subplot(2, 5, k)
  plt.imshow(output)
  plt.title(text, fontsize = 10)
  plt.grid(b=None)
  plt.axis('off')

  k += 1

plt.savefig(SAVE_PATH_CNN + '/predictions.png')

[INFORMATION] Loading Neural Network Model and Label Binarizer
